# Lagrangian Jacobian optimization

## Import statements

In [1]:
%load_ext autoreload
%autoreload 2

# 3rd party imports
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(linewidth=1000)
from scipy.sparse.linalg import lgmres
import sympy as sp


# Local imports
import modules.laplacian as laplacian
import modules.jacobian as jacobian

## Set up A $\phi$ = B case
Set up an example A*$\phi$ = B problem where there are negative Jacobian determinant values

In [2]:
# Create some example correspondence points
msample = np.array([  # Moving points
    [0, 0, 1],
    [0, 2, 1],
    [0, 2, 0],
])
fsample = np.array([  # Fixed points
    [0, 0, 2],
    [0, 2, 2],
    [0, 1, 1],
])
fixed_sample = np.zeros((1, 3, 3))  # Size of the fixed image

#############################################################################

# Create a Laplacian matrix from the sample set of correspondence points
deformation, A, Zd, Yd, Xd = laplacian.sliceToSlice3DLaplacian(fixed_sample, msample, fsample)
Adense = A.toarray()

z, y, x = (0, 1, 1)  # Preview test index
laplacian_idx = laplacian.get_laplacian_index(z, y, x, fixed_sample.shape)

# Visualize values
print()
print("A shape:", A.shape)
print("Rank of A:", np.linalg.matrix_rank(Adense))
print("A:")
print(Adense)
print()
print("Xd:", Xd)
print("Yd:", Yd)
print()

# Solving for Xd Yd
phi_x = lgmres(A, Xd, tol = 1e-2)[0]
print("phi_x:", phi_x)
phi_y = lgmres(A, Yd, tol = 1e-2)[0]
print("phi_y:", phi_y)

# Create expanded matrix to cover Xd and Yd
A0 = np.zeros((A.shape[0], A.shape[1]))
A_expanded = np.block([
    [Adense, A0],
    [A0, Adense]
])
XYd = np.concatenate([Xd, Yd])
phi_xy = lgmres(A_expanded, XYd, tol = 1e-2)[0]

print()
print("A_expanded shape:", A_expanded.shape)
print("Rank of A_expanded:", np.linalg.matrix_rank(A_expanded))
print(A_expanded)
print("XYd:", XYd)
print("phi_xy:", phi_xy)
print()

fdata.shape (1, 3, 3)
Building data for Laplacian Sparse Matrix A
Creating Laplacian Sparse Matrix A
Computing dz
dz calculated in 0.04022860527038574s
Computing dy
dy calculated in 0.05093216896057129s
Computing dx
dx calculated in 0.051349639892578125s

A shape: (9, 9)
Rank of A: 9
A:
[[ 2. -1.  0. -1.  0.  0.  0.  0.  0.]
 [-1.  3. -1.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  3. -1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0. -1.  3.  0.  0. -1.]
 [ 0.  0.  0. -1.  0.  0.  2. -1.  0.]
 [ 0.  0.  0.  0. -1.  0. -1.  3. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]

Xd: [ 0.  0. -1.  0. -1.  0.  0.  0. -1.]
Yd: [0. 0. 0. 0. 1. 0. 0. 0. 0.]

phi_x: [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
phi_y: [0.66666667 0.55555556 0.         0.77777778 1.         0.33333333 0.66666667 0.55555556 0.        ]

A_expanded shape: (18, 18)
Rank of A_expanded: 18
[[ 2. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1

/tmp/ipykernel_565021/2165156065.py:35: DeprecationWarning: scipy.sparse.linalg.lgmres called without specifying `atol`. The default value will change in the future. To preserve current behavior, set ``atol=tol``.
  phi_x = lgmres(A, Xd, tol = 1e-2)[0]
/tmp/ipykernel_565021/2165156065.py:37: DeprecationWarning: scipy.sparse.linalg.lgmres called without specifying `atol`. The default value will change in the future. To preserve current behavior, set ``atol=tol``.
  phi_y = lgmres(A, Yd, tol = 1e-2)[0]
/tmp/ipykernel_565021/2165156065.py:47: DeprecationWarning: scipy.sparse.linalg.lgmres called without specifying `atol`. The default value will change in the future. To preserve current behavior, set ``atol=tol``.
  phi_xy = lgmres(A_expanded, XYd, tol = 1e-2)[0]


In [3]:
z, y, x = (0, 0, 0) 
print("Current index:", laplacian.get_laplacian_index(z, y, x, fixed_sample.shape))
print("Adjacent indices:", laplacian.get_adjacent_indices(z, y, x, fixed_sample.shape))

Current index: 0
Adjacent indices: [None, 1, None, 3]


In [4]:
def dxx(z: int, y: int, x: int, phi: np.ndarray, shape: tuple):
    # delta x(x + 1, y, z) - delta x(x - 1, y, z) / 2
    if x == 0:  # Left edge
        right_x = phi[laplacian.get_laplacian_index(z, y, x + 1, shape)]
        curr_x = phi[laplacian.get_laplacian_index(z, y, x, shape)]
        return (right_x - curr_x) / 2
    elif x == shape[2] - 1:  # Right edge
        left_x = phi[laplacian.get_laplacian_index(z, y, x - 1, shape)]
        curr_x = phi[laplacian.get_laplacian_index(z, y, x, shape)]
        return (curr_x - left_x) / 2
    else:
        right_x = phi[laplacian.get_laplacian_index(z, y, x + 1, shape)]
        left_x = phi[laplacian.get_laplacian_index(z, y, x - 1, shape)]
        return (right_x - left_x) / 2


def dyy(z: int, y: int, x: int, phi: np.ndarray, shape: tuple):
    # delta y(x, y + 1, z) - delta y(x, y - 1, z) / 2
    if y == 0:  # Top edge
        down_y = phi[laplacian.get_laplacian_index(z, y + 1, x, shape) + len(phi) // 2]
        curr_y = phi[laplacian.get_laplacian_index(z, y, x, shape) + len(phi) // 2]
        return (down_y - curr_y) / 2
    elif y == shape[1] - 1:  # Bottom edge
        curr_y = phi[laplacian.get_laplacian_index(z, y, x, shape) + len(phi) // 2]
        up_y = phi[laplacian.get_laplacian_index(z, y - 1, x, shape) + len(phi) // 2]
        return (curr_y - up_y) / 2
    else:
        down_y = phi[laplacian.get_laplacian_index(z, y + 1, x, shape) + len(phi) // 2]
        up_y = phi[laplacian.get_laplacian_index(z, y - 1, x, shape) + len(phi) // 2]
        return (down_y - up_y) / 2


def dxy(z: int, y: int, x: int, phi: np.ndarray, shape: tuple):
    # delta x (x, y + 1, z) - delta x (x, y - 1, z) / 2
    if y == 0:  # Top edge
        down_x = phi[laplacian.get_laplacian_index(z, y + 1, x, shape)]
        curr_x = phi[laplacian.get_laplacian_index(z, y, x, shape)]
        return (down_x - curr_x) / 2
    elif y == shape[1] - 1:  # Bottom edge
        curr_x = phi[laplacian.get_laplacian_index(z, y, x, shape)]
        up_x = phi[laplacian.get_laplacian_index(z, y - 1, x, shape)]
        return (curr_x - up_x) / 2
    else:
        down_x = phi[laplacian.get_laplacian_index(z, y + 1, x, shape)]
        up_x = phi[laplacian.get_laplacian_index(z, y - 1, x, shape)]
        return (down_x - up_x) / 2


def dyx(z: int, y: int, x: int, phi: np.ndarray, shape: tuple):
    # delta y (x + 1, y, z) - delta y (x - 1, y, z) / 2
    if x == 0:  # Left edge
        right_y = phi[laplacian.get_laplacian_index(z, y, x + 1, shape) + len(phi) // 2]
        curr_y = phi[laplacian.get_laplacian_index(z, y, x, shape) + len(phi) // 2]
        return (right_y - curr_y) / 2
    elif x == shape[2] - 1:  # Right edge
        curr_y = phi[laplacian.get_laplacian_index(z, y, x, shape) + len(phi) // 2]
        left_y = phi[laplacian.get_laplacian_index(z, y, x - 1, shape) + len(phi) // 2]
        return (curr_y - left_y) / 2
    else:
        right_y = phi[laplacian.get_laplacian_index(z, y, x + 1, shape) + len(phi) // 2]
        left_y = phi[laplacian.get_laplacian_index(z, y, x - 1, shape) + len(phi) // 2]
        return (right_y - left_y) / 2


def jdet(z: int, y: int, x: int, phi: np.ndarray, shape: tuple):
    return (dxx(z, y, x, phi, shape) + 1) * (dyy(z, y, x, phi, shape) + 1) - dxy(z, y, x, phi, shape) * dyx(z, y, x, phi, shape)


## Set up Lagrangian optimizer

In [5]:
import sympy as sp

def idx_adjacency(shape):
    idx_adj = {}
    for z in range(shape[0]):
        for y in range(shape[1]):
            for x in range(shape[2]):
                l_idx = laplacian.get_laplacian_index(z, y, x, shape)
                adjacent_indices = laplacian.get_adjacent_indices(z, y, x, shape)
                idx_adj[l_idx] = adjacent_indices
    return idx_adj

def jacobian_constraint(idx, idx_adj, phi_symbols):
    # (dxx + 1) * (dyy + 1) - dxy * dyx
    y_offset = len(phi_symbols) // 2
    adj_idx = idx_adj[idx]
    dxx = (phi_symbols[adj_idx[1]] - phi_symbols[adj_idx[0]]) / 2
    dyy = (phi_symbols[adj_idx[3] + y_offset] - phi_symbols[adj_idx[2] + y_offset]) / 2
    #  delta x (x, y + 1, z) - delta x (x, y - 1, z) / 2
    dxy = (phi_symbols[adj_idx[3]] - phi_symbols[adj_idx[2]]) / 2
    dyx = (phi_symbols[adj_idx[1] + y_offset] - phi_symbols[adj_idx[0] + y_offset]) / 2
    return (dxx + 1) * (dyy + 1) - (dxy * dyx)

Define the variables

In [6]:
def generate_phi_str(b: np.ndarray): 
    phi_str = ""
    for i in range(b.shape[0]):
        if i < b.shape[0] // 2:
            phi_str += f"phi_x{i + 1}"
        else:
            phi_str += f"phi_y{(i + 1) - b.shape[0] // 2}"
        if i < b.shape[0] - 1:
            phi_str += " "
    return phi_str


def generate_mu_str(size: int):
    mu_str = ""
    for i in range(size):
        mu_str += f"mu{i + 1}"
        if i < size - 1:
            mu_str += " "
    return mu_str

phi_str = generate_phi_str(XYd)
print(phi_str)

mu_str = generate_mu_str(2)
print(mu_str)

# Add the constraint variables to the string
variable_str = phi_str + " " + mu_str

# Define the variables (phi_x1, phi_x2, phi_x3, ..., phi_y1, phi_y2, phi_y3)
# phii is a tuple containing the symbols for the variables
phi_symbols = sp.symbols(phi_str)
mu_symbols = sp.symbols(mu_str)

phi_x1 phi_x2 phi_x3 phi_x4 phi_x5 phi_x6 phi_x7 phi_x8 phi_x9 phi_y1 phi_y2 phi_y3 phi_y4 phi_y5 phi_y6 phi_y7 phi_y8 phi_y9
mu1 mu2


Define the linear system

In [7]:
# Define the matrix A and vector b
A_ = sp.Matrix(A_expanded)
b_ = sp.Matrix(XYd)

# Define the vector phi
phi_vec = sp.Matrix(phi_symbols)

Define objective function and constraints

In [74]:
# Define the objective function
objective_function = (A_ * phi_vec - b_).dot(A_ * phi_vec - b_)  # Minimize the squared Euclidean distance

# Define the constraints
#h1 = phi_symbols[0] - 2
ia = idx_adjacency(fixed_sample.shape)
print(ia)
h1 = jacobian_constraint(4, idx_adjacency(fixed_sample.shape), phi_symbols)
print(h1)
h2 = h1
h_constraints = [h1, h2]

# Construct the Lagrangian function
L = objective_function + mu_symbols[0] * h1 + mu_symbols[1] * h2

{0: [None, 1, None, 3], 1: [0, 2, None, 4], 2: [1, None, None, 5], 3: [None, 4, 0, 6], 4: [3, 5, 1, 7], 5: [4, None, 2, 8], 6: [None, 7, 3, None], 7: [6, 8, 4, None], 8: [7, None, 5, None]}
-(-phi_x2/2 + phi_x8/2)*(-phi_y4/2 + phi_y6/2) + (-phi_x4/2 + phi_x6/2 + 1)*(-phi_y2/2 + phi_y8/2 + 1)


Compute the partial derivatives

In [75]:
# Compute the partial derivatives
dL_dphi = [sp.diff(L, phi_i) for phi_i in phi_symbols]

# Compute the partial derivatives for the constraints
dL_dmu = [sp.diff(L, mu_i) for mu_i in mu_symbols]

Solve the system and display results

In [ ]:
# Solve the system of equations
#solutions = sp.solve(dL_dphi, phi_list)
sol_list = [d for d in dL_dphi] + h_constraints + [mu * h for mu, h in zip(mu_symbols, h_constraints)]
solutions = sp.solve(sol_list, phi_symbols + mu_symbols)

# Display the solutions
print("Solutions:")
if isinstance(solutions, dict):
    # Single solution case
    for i, phi_i in enumerate(phi_symbols):
        print(f"{phi_i} = {solutions[phi_i]}")
else:
    # Multiple solutions case
    for sol in solutions:
        for i, phi_i in enumerate(phi_symbols):
            print(f"{phi_i} = {sol[i]}")